# Ninety-Nine Haskell Problems

Coming from [here](https://wiki.haskell.org/H-99:_Ninety-Nine_Haskell_Problems).

### 1

 Find the last element of a list.

In [8]:
lastElement :: [a] -> a
lastElement [] = error "Empty list!"
lastElement [x] = x
lastElement (x:xs) = lastElement xs

### 2

Find the last but one element of a list.

In [9]:
lastButOneElement :: [a] -> a
lastButOneElement [] = error "Empty list!"
lastButOneElement [x] = error "Singleton!"
lastButOneElement [x, y] = x 
lastButOneElement (x:xs) = lastButOneElement xs 

### 3

Find the K'th element of a list. The first element in the list is number 1.

In [10]:
kthElement :: [a] -> Int -> a
kthElement _ 0 = error "First index is 1!"
kthElement [] _ = error "Not enough elements!"
kthElement (x:xs) 1 = x
kthElement (x:xs) n = kthElement xs (n-1)

### 4

Find the number of elements of a list.

In [11]:
numberElements :: [a] -> Int
numberElements [] = 0
numberElements (x:xs) = 1 + numberElements xs

### 5

Reverse a list.

In [26]:
reverseList :: [a] -> [a]
reverseList [] = []
reverseList (x:xs) = reverseList xs ++ [x]

### 6

Find out whether a list is a palindrome. A palindrome can be read forward or backward; e.g. (x a m a x).

In [29]:
isPalindrome x = x == reverseList x

### 7

Flatten a nested list structure.

A custom data type is needed for nested lists:

In [36]:
data NestedList a = Elem a | List [NestedList a]

In [45]:
flattenList :: NestedList a -> [a]
flattenList (Elem x) = [x]
flattenList (List (x:xs)) = (flattenList x) ++ (flattenList (List xs))

### 8

Eliminate consecutive duplicates of list elements.

In [52]:
elimConsec :: Eq a => [a] -> [a]
elimConsec [] = []
elimConsec [x] = [x]
elimConsec (x:y:xs)
    | x == y = elimConsec (x:xs)
    | otherwise = x: elimConsec (y:xs)

### 9

Pack consecutive duplicates of list elements into sublists. If a list contains repeated elements they should be placed in separate sublists.

In [4]:
packConsec :: Eq a => [a] -> [[a]]
packConsec [] = []
packConsec [x] = [[x]]
packConsec (x:xs)
    | x == (head (head (packConsec xs))) = (x:(head (packConsec xs))):(tail (packConsec xs))
    | otherwise = [x]:(packConsec xs)

### 10

Run-length encoding of a list. Use the result of problem P09 to implement the so-called run-length encoding data compression method. Consecutive duplicates of elements are encoded as lists (N E) where N is the number of duplicates of the element E.

In [15]:
runLengthEncod :: Eq a => [a] -> [(Int, a)]
runLengthEncod x =
    let packed = packConsec x
        lengths = map numberElements packed  
        heads = map head packed 
    in  zip lengths heads  

### 11
Modify the result of problem 10 in such a way that if an element has no duplicates it is simply copied into the result list. Only elements with duplicates are transferred as (N E) lists.

In [103]:
data Encoded a = Single a | Multiple (Int, a) deriving (Show, Read, Eq)  

In [104]:
toEncod :: (Int, a) -> Encoded a
toEncod (1, x) = Single x
toEncod (n, x) = Multiple (n, x)

In [55]:
runLengthEncod' :: Eq a => [a] -> [Encoded a] 
runLengthEncod' x = map toEncod $ runLengthEncod x

### 12

Decode a run-length encoded list.

In [60]:
toDecod :: Eq a => Encoded a -> [a]
toDecod (Single x) = [x]
toDecod (Multiple (0, x)) = []
toDecod (Multiple (1, x)) = toDecod (Single x)
toDecod (Multiple (n, x)) = x:(toDecod (Multiple (n-1, x)))

In [87]:
runLengthDecod :: Eq a => [Encoded a] -> [a]
runLengthDecod x = concatMap toDecod x

### 13

Run-length encoding of a list (direct solution).

Implement the so-called run-length encoding data compression method directly. I.e. don't explicitly create the sublists containing the duplicates, as in problem 9, but only count them. As in problem P11, simplify the result list by replacing the singleton lists (1 X) by X.

In [100]:
runLengthEncod'' :: Eq a => [a] -> [(Int, a)]
runLengthEncod'' [] = []
runLengthEncod'' [x] = [(1, x)]
runLengthEncod'' (x:xs)
    | x == (snd hexs) = ((fst hexs) + 1, x):texs 
    | otherwise = (1, x):encodxs
    where encodxs = runLengthEncod'' xs
          hexs = head encodxs
          texs = tail encodxs

In [107]:
runLengthEncodDirect :: Eq a => [a] -> [Encoded a]
runLengthEncodDirect x = map toEncod (runLengthEncod'' x)

### 14

Duplicate the elements of a list.

In [112]:
repElem :: Int -> a -> [a]
repElem 0 _ = []
repElem n x = x:(repElem (n-1) x)

repElems :: Int -> [a] -> [a]
repElems 0 _ = []
repElems _ [] = []
repElems n (x:xs) = (repElem n x) ++ repElems n xs 

In [116]:
duplicateElems = repElems 2

### 15

Replicate the elements of a list a given number of times.

Done above.

### 16

Drop every N'th element from a list.